# Xây dựng mô hình

Bài toán này nhóm em thử với nhiều mô hình khác nhau, bao gồm các mô hình học máy truyền thống và mô hình học sâu. 

Đối với các mô hình học máy truyền thống, nhóm em train trên các mô hình quen thuộc như: SVC, Random Forest, XgBoost, Navie Bayes và Logistic Regression.

Đối với mô hình học sâu, nhóm em sử dụng RoBERTa (Robustly Optimized BERT Approach), đây là một mô hình ngôn ngữ rất mạnh trong lĩnh vực xử lý ngôn ngữ tự nhiên (NLP)

## Đọc dữ liệu từ file sau khi tiền xử lý

(Nếu gộp 1 file thì phần này bỏ)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
train_df = pd.read_csv('train_preprocessing.csv')
test_df = pd.read_csv('test_preprocessing.csv')

In [3]:
train_df

,Unnamed: 0,id,keyword,location,text,target
0,0,1,None,None,deeds reason earthquake allah forgive,1
1,1,4,None,None,forest fire near la ronge sask canada,1
2,2,5,None,None,residents asked shelter place notified officer...,1
3,3,6,None,None,people receive wildfires evacuation orders cal...,1
4,4,7,None,None,got sent photo ruby alaska smoke wildfires pou...,1
...,...,...,...,...,...,...
7496,7604,10863,None,None,worldnews fallen powerlines glink tram update ...,1
7497,7605,10864,None,None,flip im walmart bomb evacuate stay tuned blow,1
7498,7606,10866,None,None,suicide bomber kills saudi security site mosqu...,1
7499,7608,10869,None,None,giant cranes holding bridge collapse nearby homes,1


## Feature engineering

Chúng ta chỉ sử dụng thông tin từ cột `text` để train cho mô hình. Nhóm đã thử sử dụng kết hợp cả thông tin của cột `keyword` và `location` nhưng khi chạy thì ra kết quả thấp hơn đáng kể. Kết quả trong trường hợp chỉ sử dụng duy nhất cột `text` là tốt nhất.

In [4]:
x = train_df['text']
y = train_df['target']
x1 = test_df['text']

Tiếp theo, chúng ta tiến hành chia dữ liệu có được thành các tập train và test để phục vụ cho quá trình training và testing. Dữ liệu được chia với `test_size` = 0.2

In [5]:
train_texts, test_texts, train_labels, test_labels = train_test_split(x.values, y.values, test_size=.20, random_state=42, stratify=y)

Đối với các mô hình học máy truyền thống, nhóm sử dụng **TF-IDF** để tạo đặc trưng trước khi đưa dữ liệu vào mô hình

TF-IDF là viết tắt của “Term Frequency, Inverse Document Frequency” - tạm dịch “Tần suất thuật ngữ, Tần suất tài liệu nghịch đảo”. Đó là một cách để chấm điểm tầm quan trọng của các từ (hoặc "các thuật ngữ") dựa trên tần suất xuất hiện của chúng xuất hiện trên nhiều tài liệu dựa trên quy tắc sau:

- Nếu một từ xuất hiện thường xuyên trong tài liệu, điều đó rất quan trọng. Cho từ này điểm cao.
- Nhưng nếu một từ xuất hiện trong nhiều tài liệu, thì đó không phải là mã định danh duy nhất. Cho từ đó điểm thấp.

Do đó, những từ phổ biến như `the` và `for` xuất hiện trong nhiều tài liệu sẽ được scaled down. Các từ xuất hiện thường xuyên trong một tài liệu sẽ được scaled up.

Với những giải thích trên, ta có công thức tính trọng số của một từ trong tài liệu trong ngữ liệu như sau:

$$w_{i,j} = tf_{i,j} \cdot idf_i = tf_{i,j} \cdot log(\frac {N}{df_i})$$

Trong đó:

- $tf_{i,j}$: Tần suất xuất hiện của i trong j
- $N$: Tổng số tài liệu
- $df_i$: Số tài liệu chứa i

**Reference:** https://medium.com/analytics-vidhya/an-introduction-to-tf-idf-using-python-5f9d1a343f77

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [7]:
# Sử dụng TfidfVectorizer để tạo đặc trưng
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(train_texts)
test_features = vectorizer.transform(test_texts)

## Xây dựng mô hình

Sau khi đã có các đặc trưng, việc còn lại chỉ là sử dụng các thư viện học máy như sklearn, gọi hàm rồi quăng dữ liệu vào và đợi kết quả thôi.

In [16]:
from sklearn.svm import SVC
# Huấn luyện mô hình SVC
svc_model = SVC()
svc_model.fit(train_features, train_labels)

# Dự đoán kết quả
predictions_svc = svc_model.predict(test_features)


val_accuracy = accuracy_score(predictions_svc, test_labels)
print('Validation Accuracy:', val_accuracy)

Validation Accuracy: 0.8027981345769487


In [17]:
from sklearn.ensemble import RandomForestClassifier

# Huấn luyện mô hình RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(train_features, train_labels)

# Dự đoán kết quả
predictions_rf = rf_model.predict(test_features)

val_accuracy = accuracy_score(predictions_rf, test_labels)
print('Validation Accuracy:', val_accuracy)

Validation Accuracy: 0.7841439040639574


In [18]:
import xgboost as xgb

# Huấn luyện mô hình xbg
xbg_model = xgb.XGBClassifier()
xbg_model.fit(train_features, train_labels)

# Dự đoán kết quả
predictions_xgb = xbg_model.predict(test_features)

val_accuracy = accuracy_score(predictions_xgb, test_labels)
print('Validation Accuracy:', val_accuracy)

Validation Accuracy: 0.7721518987341772


In [19]:
from sklearn.naive_bayes import MultinomialNB

# Huấn luyện mô hình Multinomial Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(train_features, train_labels)

# Dự đoán kết quả
predictions_nb = nb_model.predict(test_features)

val_accuracy = accuracy_score(predictions_nb, test_labels)
print('Validation Accuracy:', val_accuracy)

Validation Accuracy: 0.810126582278481


In [20]:
from sklearn.linear_model import LogisticRegression

# Huấn luyện mô hình lr
lr_model = LogisticRegression()
lr_model.fit(train_features, train_labels)

# Dự đoán kết quả
predictions_lr = lr_model.predict(test_features)

val_accuracy = accuracy_score(predictions_lr, test_labels)
print('Validation Accuracy:', val_accuracy)

Validation Accuracy: 0.8061292471685543


**Nhận xét:**
- So sánh kết của giữa các mô hình thì có vẻ Random Forest và XgBoost kết quả có hơi bị thọt so với các mô hình còn lại. 
- Có vẻ như mô hình dựa trên việc sử dụng Decision Tree không tốt lắm (tệ) trong bài toán text classification. 
- Các mô hình chạy ra kết quả cũng tốt thật đấy, nhưng cũng chỉ tầm được mức 0.8 là cao nhất. Mục tiêu của nhóm là phải lọt được top 10%. Kết quả của các mô hình trên chỉ giúp đạt được top 50% (quá tệ), thế nên cần lựa chọn một mô hình tốt hơn cho bài toán này nếu muốn kết quả tốt hơn.

Nhóm cũng đã thử tìm cách để cải thiện kết quả của các mô hình truyền thống, như điều chỉnh lại 1 số chỗ ở phần tiền xử lý dữ liệu, điều chỉnh tham số mô hình,... Nhưng kết quả vẫn không thay đổi đáng kể 1 tý nào.

Sau khi tìm hiểu "1 ít" về lĩnh vực xử lý ngôn ngữ tự nhiên, nhóm phát hiện ra rằng các mô hình nổi bật trong xử lý ngôn ngữ tự nhiên đặc biệt là với task text classification đa số đều là các mô hình học sâu, trong đó cái tên xuất hiện nhiều nhất là BERT.

Ý tưởng để giải quyết bài toán là sử dụng mô hình đã pre-train từ một tập dữ liệu rất rất lớn trước, sau đó điều chỉnh tham số của mô hình (fine-tuning) rồi train trên downstream task (tức là bài toán của chúng ta). Việc mô hình ngôn ngữ đã pre-train thực sự đem lại hiệu quả rất lớn trong việc xử lý các tác vụ của xử lý ngôn ngữ tự nhiên.

BERT về cơ bản là một kiến trúc transformer được đào tạo để học cách biểu diễn ngôn ngữ. Nó chủ yếu khác với những mô hình ngôn ngữ trước đây là do khả năng hiểu được ngữ cảnh từ cả 2 phía của câu (trái và phải của chính từ đó) thay vì chỉ sử dụng thông tin từ trái sang phải (left-to-right) hoặc ngược lại. Từ đó, cho phép nắm bắt được sự phụ thuộc và ý nghĩa toàn diện hơn trong câu.

Sau khi thử BERT và các mô hình khác nhau được xây dựng dựa trên BERT, nhóm em đã thành công chọn một mô hình tốt nhất đó là RoBERTa.

RoBERTa tương tự như BERT về kiến trúc, nhưng nó đã trải qua một số cải tiến trong quá trình huấn luyện. Mục tiêu của RoBERTa là cải thiện hiệu suất so với BERT thông qua việc tinh chỉnh quá trình huấn luyện và sử dụng các siêu tham số khác nhau.

Một số khác biệt giữa RoBERTa và BERT:

- **Kích thước dữ liệu huấn luyện:** RoBERTa được tiền huấn luyện với lượng dữ liệu lớn hơn BERT gấp 10 lần (160GB và 16GB). Điều này giúp RoBERTa học được nhiều đặc trưng ngôn ngữ phổ quát hơn và cải thiện khả năng hiểu ngữ nghĩa.
- **Mặt nạ:** BERT được đào tạo với mặt nạ tĩnh, nghĩa là mặt nạ được khởi tạo 1 lần và được đưa vào huấn luyện trong mỗi epoch. Để có sự đa dạng hơn trong dữ liệu đào tạo cho mỗi epoch thì RoBERTa được áp dụng mặt nạ động. Hiệu suất của mô hình được đào tạo với mặt nạ động tốt hơn một chút hoặc ít nhất là tương đương với phương pháp ban đầu được sử dụng mặt nạ tĩnh.
- **Loại bỏ NSP:** RoBERTa không cần phải giới hạn đầu vào theo cặp câu và có thể sử dụng các cặp câu ngẫu nhiên để tăng đa dạng dữ liệu huấn luyện.

In [27]:
from transformers import RobertaTokenizer
from transformers import TFRobertaForSequenceClassification
import tensorflow as tf

2023-06-29 19:02:45.693708: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 19:02:45.717498: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 19:02:45.823718: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 19:02:45.824933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 19:02:46.464104: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [71]:
train_texts, test_texts, train_labels, test_labels = train_test_split(x.values, y.values, test_size=.20, random_state=42, stratify=y)

Nhóm em sử dụng RoBERTa cho bài toán này bằng việc import `TFRobertaForSequenceClassification` trong thư viện `transformers`.

Việc sử dụng mô hình từ thư viện này rất đơn giản, chỉ cần khởi tạo mô hình đã Pre-training với RoBERTa Base.

Sau khi khởi tạo mô hình, ta sẽ Encoding input cho phù hợp với đầu vào của mô hình (đây có thể xem như là bước tạo đặc trưng cho mô hình). 

Tiếp theo sẽ là bước tinh chỉnh mô hình (fine-tuning) và train mô hình. Sau nhiều lần thử với các tham số khác nhau, nhóm em chọn learning rate (adam) là 3e-5, batch size là 32, và epoch là 2 thì sẽ có được kết quả tốt nhất. 

Cuối cùng là evaluate model trên dữ liệu test để thu được kết quả và đánh giá

Khởi tạo mô hình và Tokenizer

In [28]:
# Khởi tạo mô hình và Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Encoding input cho phù hợp với đầu vào của mô hình 

In [74]:
#Encode train input
encoded_inputs = tokenizer(train_texts.tolist(), padding=True, truncation=True, return_tensors='tf')
labels = tf.constant(train_labels.tolist())

Điều chỉnh tham số cho mô hình

In [75]:
# Define the training parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

Compile và train model

In [76]:
# Compile and train the model
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.fit(encoded_inputs.input_ids, labels, epochs=2, batch_size=32)

Epoch 1/2
188/188 [==============================] - 157s 585ms/step - loss: 0.5338 - accuracy: 0.7319
Epoch 2/2
188/188 [==============================] - 107s 570ms/step - loss: 0.3933 - accuracy: 0.8341


Đánh giá kết quả của model

In [78]:
#Encode test input
test_inputs = tokenizer(test_texts.tolist(), padding=True, truncation=True, return_tensors='tf')
#labels = tf.constant(test_labels.tolist())


model.evaluate(
    x=test_inputs.input_ids,
    y=test_labels,
    batch_size=32
)

47/47 [==============================] - 11s 173ms/step - loss: 0.3853 - accuracy: 0.8448


[0.38526058197021484, 0.8447701334953308]

Kết quả của mô hình đạt được rất tốt, vượt trội rất nhiều so với các mô hình truyền thống. Với kết quả này nhóm tự tin đạt vào top 10%

Vì mô hình này tốt nhất nên nhóm sử dụng mô hình này để thực hiện việc dự đoán trên tập test và nộp kết quả đạt được lên Kaggle

In [79]:
#Encode predict input
predict_inputs = tokenizer(x1.tolist(), padding=True, truncation=True, return_tensors='tf')
# Make predictions
predictions = model.predict(predict_inputs.input_ids)

# Extract predicted labels
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

102/102 [==============================] - 21s 171ms/step


In [80]:
submission = pd.DataFrame({'id': test_df['id'], 'target': predicted_labels})
submission.to_csv('submission.csv', index=False)

Kết quả nhóm đạt dược: 0.